In [1]:
# Importações necessárias
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import pandas as pd

In [2]:
# Configurações de estilo profissional
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'DejaVu Sans'  # Fonte mais legível

In [3]:
# Simulação de Dados Genéticos (com mais variação)
def generate_genetic_data():
    np.random.seed(42)
    positions = np.arange(1, 101)
    mutations = np.random.choice(['SNP', 'DEL', 'INS', 'CNV'], 100, p=[0.6, 0.2, 0.15, 0.05])
    impact = np.abs(np.random.normal(0.5, 0.2, 100))
    
    # Adicionar hotspots genéticos
    impact[30:35] = np.random.uniform(0.7, 0.9, 5)
    impact[70:75] = np.random.uniform(0.7, 0.9, 5)
    
    return pd.DataFrame({
        'Position': positions,
        'Mutation': mutations,
        'Impact': impact,
        'Gene': ['GENE'+str(i%5+1) for i in range(100)]
    })

df = generate_genetic_data()

# Mapeamento dos tipos de mutação para números
mutation_map = {'SNP': 0, 'DEL': 1, 'INS': 2, 'CNV': 3}
df['MutationType'] = df['Mutation'].map(mutation_map)

In [4]:
# Configuração da Figura 3D
fig = plt.figure(figsize=(12, 8), dpi=100)
ax = fig.add_subplot(111, projection='3d')

# Paleta de cores acessível (Daltonian-friendly)
palette = {
    'SNP': '#3575D5',  # Azul
    'DEL': '#E64C4C',  # Vermelho
    'INS': '#4CAF50',  # Verde
    'CNV': '#9C27B0'   # Roxo
}

plt.close()

In [5]:
# Função de Inicialização
def init():
    ax.clear()
    ax.set_xlim(0, 100)
    ax.set_ylim(-0.5, 3.5)  # Ajuste para o mapeamento das mutações
    ax.set_zlim(0, 1.05)
    ax.set_xlabel('Posição no Genoma', fontsize=10, fontweight='bold')
    ax.set_ylabel('Tipo de Mutações', fontsize=10, fontweight='bold')
    ax.set_zlabel('Impacto Genético', fontsize=10, fontweight='bold')
    ax.set_title('Análise Genômica 3D Progressiva', fontsize=14, fontweight='bold')

    # Configurar a legenda fora da área do gráfico
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10) 
               for color in palette.values()]
    labels = list(palette.keys())
    ax.legend(handles, labels, loc='center left', fontsize=10, title="Tipos de Mutação", bbox_to_anchor=(1.05, 0.5))

    return []

In [6]:
# Função de Atualização (mais lenta e explicativa)
def update(frame):
    current_frame = min(frame * 2, 100) if frame < 50 else 100
    current_data = df.iloc[:current_frame]
    
    ax.clear()
    
    # Configurações fixas
    ax.set_xlim(0, 100)
    ax.set_ylim(-0.5, 3.5)  # Ajuste para o mapeamento das mutações
    ax.set_zlim(0, 1.05)
    
    ax.set_xlabel('Posição no Genoma', fontsize=10, fontweight='bold')
    ax.set_ylabel('Tipo de Mutações', fontsize=10, fontweight='bold')
    ax.set_zlabel('Impacto Genético', fontsize=10, fontweight='bold')
    ax.set_title(f'Análise Genômica 3D Progressiva (Frame {current_frame})', fontsize=14, fontweight='bold')
    
    # Plotar mutações como pontos no gráfico 3D
    for mut_type, group in current_data.groupby('Mutation'):
        mut_num = mutation_map[mut_type]  # Usar o número mapeado para o tipo de mutação
        ax.scatter(group['Position'], [mut_num] * len(group), group['Impact'], 
                   color=palette[mut_type], label=mut_type, s=80, alpha=0.7, edgecolors='w', linewidth=0.5)
    
    # Linha de progresso e hotspot markers
    if current_frame >= 30:
        ax.text(30, 0.5, 0.7, "Hotspot 1", color="black")
    if current_frame >= 70:
        ax.text(70, 0.5, 0.7, "Hotspot 2", color="black")

    # Animação suave nos eixos
    ax.view_init(elev=20, azim=current_frame * 2)  # Alterar o ângulo de visão com base no frame
    
    # Re-adicionar a legenda a cada frame
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10) 
               for color in palette.values()]
    labels = list(palette.keys())
    ax.legend(handles, labels, loc='center left', fontsize=10, title="Tipos de Mutação", bbox_to_anchor=(1.05, 0.5))

    return []

In [8]:
# Criar Animação com velocidade acessível
ani = FuncAnimation(fig, update, frames=60, init_func=init, interval=400, blit=False)

# Salvar com qualidade otimizada
ani.save('../media/genome_analysis_3d_with_legend.gif',
          writer='pillow',
          fps=5, # Reduzido para melhor acompanhamento
          dpi=100,
          progress_callback=lambda i, n: print(f'Processando frame {i}/{n}') if i%10==0 else None)

plt.close()

print("\nAnimação finalizada! Salva como '../media/genome_analysis_3d_with_legend.gif'")

Processando frame 0/60
Processando frame 10/60
Processando frame 20/60
Processando frame 30/60
Processando frame 40/60
Processando frame 50/60

Animação finalizada! Salva como '../media/genome_analysis_3d_with_legend.gif'
